In [1]:
#Import libraries
#Set seed for random number generator to create reproducible results
library(ISLR)
set.seed(1)
train=sample(392,196)

Warning message:
"package 'ISLR' was built under R version 3.6.1"

In [2]:
#?sample

In [3]:
#Use subset option in LM to fit regression using only training set
lm.fit=lm(mpg~horsepower,data=Auto,subset=train)

In [4]:
#Use predict function to estimate response for 392 observ
#Mean function to calculate MSE of 196 observations
# "negative train" selects observations not in the training set
attach(Auto)
mean((mpg-predict(lm.fit,Auto))[-train]^2)

[1] 23.26601

<b>Mean Squared Error = 23.26<b>

In [5]:
# poly() function to estimate  test error for quadratic and cubic regression
lm.fit2=lm(mpg~poly(horsepower,2), data=Auto,subset=train)
mean((mpg-predict(lm.fit2,Auto))[-train]^2)

[1] 18.71646

In [6]:
lm.fit3=lm(mpg~poly(horsepower,3),data=Auto,subset=train)
mean((mpg-predict(lm.fit3,Auto))[-train]^2)

[1] 18.79401

Error rates are 19.82 and 19.78.

Choosing a different training set will result in different errors on thevalidation set.

In [7]:
#Linear Model
set.seed(2)
train=sample(392,196)
lm.fit=lm(mpg~horsepower,subset=train)
mean((mpg-predict(lm.fit,Auto))[-train]^2)

[1] 25.72651

In [8]:
#Cubic Model
lm.fit2=lm(mpg~poly(horsepower,2),data=Auto,subset=train)
mean((mpg-predict(lm.fit2,Auto))[-train]^2)

[1] 20.43036

In [9]:
#Quadratic Model
lm.fit3=lm(mpg~poly(horsepower,3),data=Auto,subset=train)
mean((mpg-predict(lm.fit3,Auto))[-train]^2)

[1] 20.38533

Quadratic function performs better than linear regression in predicting Horsepower by MPG in the Auto dataset.

## Leave-One-Out Cross-Validation

Using Generalized Linear Model ("glm") without the (family="binomial") will yield Linear Regression

In [10]:
glm.fit=glm(mpg~horsepower,data=Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [11]:
#Results ame as above
lm.fit=lm(mpg~horsepower,data=Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [12]:
#glm() can be used with Cross Validation (cv) from the "boot" library
library(boot)
glm.fit=glm(mpg~horsepower,data=Auto)
cv.err=cv.glm(Auto,glm.fit)
cv.err$delta

[1] 24.23151 24.23114

The 2 numbers in the delta vector above contain cross-validation results. This corresponds to the LOOCV statistic.

In [13]:
#Use for loop to fit polynomial regression i=1 to i=5
#Stores CV Error and stores it in the "ith" element of vector "cv.error"
cv.error=rep(0,5)
for (i in 1:5){
    glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
    cv.error[i]=cv.glm(Auto,glm.fit)$delta[1]
    }
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

<b>Note the sharp drop in estimated test MSE between linear and quadratic fits.<b>

## k-Fold Cross-Validation

<b>5 and 10 k-Folds are standard practice<b>

In [14]:
#cv.glm() to implement k-Fold CV
#Use for loop to fit polynomial regression i=1 to i=10
#Stores CV Error and stores it in the "ith" element of vector "cv.error"
set.seed(17)
cv.error.10=rep(0,10)
for (i in 1:10){
    glm.fit=glm(mpg~poly(horsepower,i),data=Auto)
    cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1]
    }
cv.error.10

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

<b>Note the model does not improve with a cubic polynomial and beyond<b>

## Bootstrap

Two Steps to Bootstrap
1. Create a function to compute statistic of interest
2. Use boot() function to perform bootstrap that repeatedly samples observations from the data set with replacement

In [15]:
#Create function alpha.fn() inputs (X,Y)
#Creates vector to determine observations used to estimate Alpha
alpha.fn=function(data,index){
    X=data$X[index]
    Y=data$Y[index]
    return((var(Y)-cov(X,Y))/var(X)+var(Y)-2*cov(X,Y))
    }

In [16]:
#Outputs estimate of Alpha based on the observations indexed
#To estimate Alpha using all 100 observations
alpha.fn(Portfolio,1:100)

[1] 0.6596797

In [17]:
#Use sample function to randomly select 100 observations
#Thus creating new bootstrap dataset and recomputing Alpha-hat
set.seed(1)
alpha.fn(Portfolio,sample(100,100,replace=T))

[1] 1.077955

In [18]:
#Boot function to automate process
boot(Portfolio,alpha.fn,R=1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.6596797 0.0009365922   0.3082678

Using original data, Alpha-hat=0.6596 and bootstrap estimate for SE=0.3082

## Estimate Accuracy of Linear Regression Model

In [20]:
#Apply function to full set of 392 observations
boot.fn=function(data,index)
    return(coef(lm(mpg~horsepower,data=data,subset=index)))
boot.fn(Auto,1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [21]:
#Create boostrap estimates by random sample with obsv replacement
set.seed(1)
boot.fn(Auto,sample(392,392,replace=T))

(Intercept)  horsepower 
 40.3404517  -0.1634868

In [22]:
#Use boot function to compute SE of 1,000 bootstrap estimates
boot(Auto,boot.fn,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0549915227 0.841925746
t2* -0.1578447 -0.0006210818 0.007348956

In [23]:
#Summary function to compute SE for regression coefficients
summary(lm(mpg~horsepower,data=Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81


Intercept= 0.71
Slope= 0.0064

In [24]:
boot.fn=function(data,index)
    coefficients(lm(mpg~horsepower+I(horsepower^2),data=data,
                   subset=index))
set.seed(1)
boot(Auto,boot.fn,1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164